### Authenticate and get headers

In [ ]:
# https://help.tableau.com/current/api/rest_api/en-us/REST/TAG/index.html#tag/Pulse-Methods/operation/MetricQueryService_CreateDefinition

In [ ]:
import requests
from _includes import config
from _includes import helper

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### Prepare to Create Definition

In [ ]:
datasource_id = '3c2b0b36-6b89-449e-9496-951e1a31eb53'
pulse_name = 'Number of Visitor 03'
pulse_description = 'Number of visitor of sample store in a company'

### Measure Customization

In [ ]:
# Customized Formula
formula = 'SUM([NumberOfVisit]) / AVG([NumberOfVisit])'
is_running_total = False

# Time Dimension and Filters
time_dimension_field = 'Month'
list_allowed_dimension = ['Branch']

In [ ]:
# GRANULARITY_BY_YEAR / GRANULARITY_BY_QUARTER / GRANULARITY_BY_MONTH / GRANULARITY_BY_WEEK / GRANULARITY_BY_DAY
list_allowed_granularity = []

# NUMBER_FORMAT_TYPE_NUMBER / NUMBER_FORMAT_TYPE_PERCENT / NUMBER_FORMAT_TYPE_CURRENCY
number_format = 'NUMBER_FORMAT_TYPE_NUMBER'

# SENTIMENT_TYPE_NONE / SENTIMENT_TYPE_UP_IS_GOOD / SENTIMENT_TYPE_DOWN_IS_GOOD
sentiment_type = 'SENTIMENT_TYPE_UP_IS_GOOD'

# NOUNS
singular_noun = ''
plural_noun = ''

### Create a JSON payload

In [ ]:
payload = {
    'name': pulse_name,
    'description': pulse_description,
    'specification': {
        'datasource': {
            'id': datasource_id
        },
        'viz_state_specification': {
            'viz_state_string': (
                '{"vizState":{"rows":[{"fieldOnShelf":{"component":["Calculation_Pulse"]},"fieldCaption":"Calculation_Pulse"}],'
                '"columns":[{"fieldOnShelf":{"component":["'+time_dimension_field+'"]},"fieldCaption":"'+time_dimension_field+'"}]},'
                '"dataModel":{"dataSource":[{"columnsToAdd":[{"name":{"component":["Calculation_Pulse"]},"fieldType":"FIELD_TYPE_CONTINUOUS",'
                '"pivotStrategy":"FIELD_PIVOT_STRATEGY_PIVOT_ON_KEY","role":"FIELD_ROLE_MEASURE","dataType":"DATA_TYPE_REAL_TYPE",'
                '"calc":{"formula":"'+formula+'"}}]}]}}'
            )
        },
        'is_running_total': is_running_total
    },
    'extension_options': {
        'allowed_dimensions': list_allowed_dimension,
        'allowed_granularities': list_allowed_granularity,
        'offset_from_today': 0
    },
    'representation_options': {
        'type': number_format,
        'number_units': {
            'singular_noun': singular_noun,
            'plural_noun': plural_noun
        },
        'sentiment_type': sentiment_type
    },
    'comparisons': {
        'comparisons': [
            {
                'compare_config': {
                    'comparison': 'TIME_COMPARISON_PREVIOUS_PERIOD'
                },
                'index': 0
            },
            {
                'compare_config': {
                    'comparison': 'TIME_COMPARISON_YEAR_AGO_PERIOD'
                },
                'index': 1
            }
        ]
    }
}

### Post to Pulse Definitions Endpoint

In [ ]:
endpoint = f'{base_api_url}/pulse/definitions'

try:
    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()
    print('Metric definition created successfully.')
    j = response.json()
    definition_id = j['definition']['metadata']['id']
    print(f'Metric Definition ID:')
    print(definition_id)
except requests.exceptions.RequestException as e:
    print(f'Request failed: {e}')
    if response.content:
        print('Response content:', response.content.decode())